In [1]:
import pandas as pd

FILENAME = 'vicuna_benchmark_human_annotations.csv'
df = pd.read_csv(FILENAME)

questions_df = pd.read_json('../../prompts/vicuna_questions.jsonl', lines=True)

In [6]:
questions_df.head()

,question_id,text,category
0,1,How can I improve my time management skills?,generic
1,2,What are the most effective ways to deal with ...,generic
2,3,What are the main differences between Python a...,generic
3,4,How can I increase my productivity while worki...,generic
4,5,Can you explain the basics of quantum computing?,generic


In [2]:
merged = pd.merge(questions_df, df, left_on='text', right_on='prompt', how='outer')
merged.question_id.isna().sum()

0

In [3]:
del merged['prompt']
merged.rename(columns={'answer':'score', 'system_a': 'model_a', 'system_b': 'model_b'}, inplace=True)
merged.score = merged.score.map({'system_a': 1, 'system_b': 2, 'tie': 3})

In [7]:
merged.head(3)
merged

0        guanaco-7b
1        guanaco-7b
2        guanaco-7b
3       guanaco-13b
4       guanaco-13b
           ...     
6715    guanaco-65b
6716    guanaco-65b
6717          gpt35
6718          gpt35
6719          gpt35
Name: model_a, Length: 6720, dtype: object

In [4]:
# get list of models in merged
modelsA = merged.model_a.unique()
modelsB = merged.model_b.unique()

for modelA in modelsA:
    for modelB in modelsB:
        if modelA == modelB:
            continue
        filtered_df = merged[(merged['model_a'] == modelA) & (merged['model_b'] == modelB)]
        
        filtered_df = filtered_df[['question_id', 'text', 'score']]

        name = f'{modelA}-vs-{modelB}-human-reviewer_threeclass.jsonl'
        filtered_df.to_json(f'../{name}', orient='records', lines=True)